In [1]:
import langchain_groq
print("langchain_groq module is installed successfully!")

langchain_groq module is installed successfully!


In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)
     

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, ret

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("SentenceTransformer Model Loaded Successfully")

SentenceTransformer Model Loaded Successfully


In [5]:
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key="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",  
        model_name="llama-3.3-70b-versatile"
    )

def create_vector_db():
    loader = DirectoryLoader("C:\\Users\\sudip\\OneDrive\\hackathon\\data", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
    vector_db.persist()
    
    print("ChromaDB created and data saved")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    
    prompt_template = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    return qa_chain

def main():
    print("Initializing Chatbot...")
    llm = initialize_llm()

    db_path = "C:\\Users\\sudip\\OneDrive\\hackathon\\data"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself. Goodbye!")
            break
        response = qa_chain.run(query)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    main()


Initializing Chatbot...


C:\Users\sudip\AppData\Local\Temp\ipykernel_4660\3961907067.py:60: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\sudip\AppData\Local\Temp\ipykernel_4660\3961907067.py:61: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, e


Human: Hello


C:\Users\sudip\AppData\Local\Temp\ipykernel_4660\3961907067.py:70: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot: Hello! It's nice to meet you. I'm here to listen and support you in a safe and non-judgmental space. How are you feeling today? Is there something on your mind that you'd like to talk about, or would you just like some company and a chance to unwind? I'm here to help in any way I can.

Human: I am so depressed
Chatbot: I'm so sorry to hear that you're feeling depressed. It takes a lot of courage to acknowledge and share your emotions, and I'm here to listen and support you without judgment. Depression can be a really tough and isolating experience, but please know that you're not alone. I'm here to offer a safe and non-judgmental space for you to express yourself.

Can you tell me more about what's been going on and how you've been feeling lately? What are some of the thoughts, emotions, or experiences that have been weighing on your mind and heart? Sometimes talking about it can help clarify things and give us a better understanding of what you're going through.

Also, I want

In [6]:
msg = {"files": ["file1.txt", "file2.txt"]} 

for x in msg.get("files", []):  
    print(x) 


file1.txt
file2.txt


In [7]:
print("Type of msg:", type(msg))  
print("Value of msg:", msg)  

for x in msg.get("files", []):  
    print(x)


Type of msg: <class 'dict'>
Value of msg: {'files': ['file1.txt', 'file2.txt']}
file1.txt
file2.txt


In [8]:
{
    "role": "assistant",
    "content": "Your response here",
    "files": []
}


{'role': 'assistant', 'content': 'Your response here', 'files': []}

In [9]:
def chat_function(input_text, history):
    response = some_chat_model(input_text)
    print("Response Type:", type(response))  
    return {"role": "assistant", "content": response, "files": []}  


In [10]:
print("Type of msg:", type(msg))  
print("Value of msg:", msg)  


Type of msg: <class 'dict'>
Value of msg: {'files': ['file1.txt', 'file2.txt']}


In [11]:
if isinstance(msg, tuple):
    msg = msg[0]  


In [12]:
for x in msg.get("files", []):  
    print(x)


file1.txt
file2.txt


In [13]:
def my_function():
    return {"text": "Hello!", "files": ["file1.txt", "file2.txt"]}

result = my_function()
print(result)


{'text': 'Hello!', 'files': ['file1.txt', 'file2.txt']}


In [14]:
def chatbot_response(message, history):
    return {"text": "Hello!", "files": ["file1.txt", "file2.txt"]} 

In [27]:
import os
import gradio as gr
import random
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  Initialize LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

#  Create Vector Database
def create_vector_db():
    loader = DirectoryLoader("C:/Users/sudip/OneDrive/hackathon/data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

#  Setup QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    
    prompt_template = """
    You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot:
    """
    
    PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

#  Recommendations Data
recommendations = {
    "music": [
        "Weightless - Marconi Union", "Someone Like You - Adele", "Fix You - Coldplay", "Rise Up - Andra Day",
        "Heal the World - Michael Jackson", "A Thousand Years - Christina Perri", "Let It Be - The Beatles",
        "Unwritten - Natasha Bedingfield"
    ],
    "yoga": [
        "Child’s Pose - For relaxation", "Cat-Cow Stretch - For gentle movement", "Legs Up the Wall Pose - To reduce stress",
        "Corpse Pose - For deep relaxation", "Seated Forward Bend - To calm the mind", "Butterfly Pose - To release tension"
    ],
    "places": [
        "Quiet Beach - For soothing waves", "Botanical Garden - To connect with nature", "Forest Cabin - Peaceful and secluded",
        "Zen Garden - Mindfulness and meditation", "Countryside Retreat - Away from stress", "Hot Springs - Relaxation through warm water",
        "Art Museum - For calming inspiration", "Library - Quiet and peaceful environment"
    ],
}

#  Get Random Suggestions
def get_suggestion(category, count=1):
    return random.sample(recommendations[category], min(count, len(recommendations[category])))

#  Initialize Components
print("Initializing Chatbot...")
llm = initialize_llm()

if not os.path.exists("./chroma_db"):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

#  Store User Questions
def log_user_question(question):
    with open("user_questions.log", "a") as log_file:
        log_file.write(question + "\n")

#  Chatbot Response Function
def chatbot_response(user_input, history=None):
    if history is None:
        history = []  # Ensure history is a list
    
    if not user_input or not isinstance(user_input, str) or not user_input.strip():
        return " Please provide a valid input."
    
    log_user_question(user_input)  # Log user question
    
    # Handle greetings
    greetings = ["hello", "hi", "hey", "hola", "namaste"]
    if user_input.lower() in greetings:
        return "👋 Hello! How can I assist you today?"
    
    # Handle suggestion requests dynamically
    suggestion_keywords = {
        "music": ["suggest music", "music recommendation", "recommend a song", "give me a song"],
        "yoga": ["suggest yoga", "yoga pose recommendation", "recommend a yoga pose", "give me a yoga pose"],
        "places": ["suggest a place", "place to visit", "recommend a location", "where should i go"]
    }

    for category, keywords in suggestion_keywords.items():
        if any(keyword in user_input.lower() for keyword in keywords):
            suggestion = get_suggestion(category, 3 if category == "places" else 1)
            return f"🌟 **{category.capitalize()} Suggestion:** {', '.join(suggestion)}"

    # Get chatbot response for general queries
    response = qa_chain.run(user_input)

    return response

#  Gradio UI with Improved Styling
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
        <div style="text-align: center; padding: 20px;">
            <h1 style="color: #4A90E2;">🧠 Mental Health Chatbot 🎵🧘‍♂️🌍</h1>
            <p style="font-size: 18px;">Welcome! Chat with our wellness assistant and get music, yoga, or place recommendations to help you relax.</p>
        </div>
    """)
    
    with gr.Row():
        with gr.Column():
            chatbot = gr.ChatInterface(fn=chatbot_response, title="💬 Wellness Chatbot")
    
    with gr.Row():
        with gr.Column():
            music_btn = gr.Button("🎵 Get Music Suggestion", variant="primary")
            music_output = gr.Textbox(label="Music Suggestion", interactive=False)
            music_btn.click(lambda: get_suggestion("music"), outputs=music_output)
        
        with gr.Column():
            yoga_btn = gr.Button("🧘 Get Yoga Suggestion", variant="primary")
            yoga_output = gr.Textbox(label="Yoga Suggestion", interactive=False)
            yoga_btn.click(lambda: get_suggestion("yoga"), outputs=yoga_output)
        
        with gr.Column():
            places_btn = gr.Button("🌍 Get Place Suggestion", variant="primary")
            places_output = gr.Textbox(label="Place Suggestion", interactive=False)
            places_btn.click(lambda: get_suggestion("places", 3), outputs=places_output)
    
app.launch(share=True)

Initializing Chatbot...


C:\Users\sudip\anaconda3\Lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7886
* Running on public URL: https://a573d2b563e710e0c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
